# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 9
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr9.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr9.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 245 245

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000107099,ENSG00000080503,ENSG00000147852,ENSG00000107249,ENSG00000147853,ENSG00000096968,ENSG00000107014,ENSG00000120217,ENSG00000197646,ENSG00000178445
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000107099,1.00000000,0.027098800,-0.06273439,-0.094130444,0.02692886,0.02801359,-0.072436548,-0.001007194,0.00000000,0.00000000
ENSG00000080503,0.02709880,1.000000000,-0.04390768,0.003128435,0.02638727,-0.06732638,-0.037079488,-0.013919870,0.03819637,-0.01490507
ENSG00000147852,-0.06273439,-0.043907679,1.00000000,-0.027363007,-0.03211562,-0.03626737,-0.029817275,-0.067241235,0.01200454,0.05222454
ENSG00000107249,-0.09413044,0.003128435,-0.02736301,1.000000000,0.02136621,0.05914285,-0.008832756,-0.048457855,0.07372137,0.01881659
ENSG00000147853,0.02692886,0.026387275,-0.03211562,0.021366208,1.00000000,0.04676392,0.107088101,0.016461678,0.18127560,0.06369230
ENSG00000096968,0.02801359,-0.067326383,-0.03626737,0.059142849,0.04676392,1.00000000,-0.056621742,0.035003523,0.07063224,0.01039246


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 19

[1] -0.007888203 -0.011034525 -0.067363727 -0.143440241 -0.153430577
 [6] -0.228470011 -0.231898927 -0.232534857 -0.308288256 -0.334113486
[11] -0.596709506 -0.696951600 -0.750523668 -0.838264324 -0.931790957
[16] -1.277399620 -1.655910497 -1.820500384 -2.582931806

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 245 245

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000107099,ENSG00000080503,ENSG00000147852,ENSG00000107249,ENSG00000147853,ENSG00000096968,ENSG00000107014,ENSG00000120217,ENSG00000197646,ENSG00000178445
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000107099,1.00000000,0.027098800,-0.06273439,-0.094130444,0.02692886,0.02801359,-0.072436548,-0.001007194,0.00000000,0.00000000
ENSG00000080503,0.02709880,1.000000000,-0.04390768,0.003128435,0.02638727,-0.06732638,-0.037079488,-0.013919870,0.03819637,-0.01490507
ENSG00000147852,-0.06273439,-0.043907679,1.00000000,-0.027363007,-0.03211562,-0.03626737,-0.029817275,-0.067241235,0.01200454,0.05222454
ENSG00000107249,-0.09413044,0.003128435,-0.02736301,1.000000000,0.02136621,0.05914285,-0.008832756,-0.048457855,0.07372137,0.01881659
ENSG00000147853,0.02692886,0.026387275,-0.03211562,0.021366208,1.00000000,0.04676392,0.107088101,0.016461678,0.18127560,0.06369230
ENSG00000096968,0.02801359,-0.067326383,-0.03626737,0.059142849,0.04676392,1.00000000,-0.056621742,0.035003523,0.07063224,0.01039246


,ENSG00000107099,ENSG00000080503,ENSG00000147852,ENSG00000107249,ENSG00000147853,ENSG00000096968,ENSG00000107014,ENSG00000120217,ENSG00000197646,ENSG00000178445
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000107099,1.0000000000,0.023726005,-0.030421178,-0.034444838,-0.0004550003,0.01106976,-0.031475617,0.005586249,0.001109964,-0.002473714
ENSG00000080503,0.0237260048,1.000000000,-0.007374758,-0.002320276,0.0236652657,-0.02073776,-0.006962422,-0.012007726,0.011957597,-0.013373576
ENSG00000147852,-0.0304211775,-0.007374758,1.000000000,0.006055768,-0.0195309952,-0.02511398,-0.014610909,-0.028170224,-0.011362604,0.011299695
ENSG00000107249,-0.0344448384,-0.002320276,0.006055768,1.000000000,0.0269569405,0.03023526,-0.002566213,-0.007391932,0.035076681,0.010279749
ENSG00000147853,-0.0004550003,0.023665266,-0.019530995,0.026956940,1.0000000000,0.02229364,0.036409737,0.062512050,0.093616523,0.048330347
ENSG00000096968,0.0110697621,-0.020737764,-0.025113982,0.030235261,0.0222936376,1.00000000,-0.019612851,0.026863488,0.045336456,0.031954543


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)